In [2]:
import pandas as pd
from tensorflow import keras

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, \
finetune, evaluate_by_len
from proteinbert.finetuning import encode_train_and_valid_sets
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
from os import path
import pickle

In [3]:
import wandb
from wandb.keras import WandbCallback

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [5]:
DATA_DIR = "../../data/"

In [6]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [7]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [8]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [9]:
wandb.init(project=f"OutOfTheBox", entity="kvetab")

wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [10]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
valid_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"), index_col=0)
train_data.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [11]:
train_data["seq"] = train_data["heavy"] + train_data["light"]
valid_data["seq"] = valid_data["heavy"] + valid_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

In [12]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True),
    WandbCallback()
]

In [13]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-4

In [14]:
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }

In [15]:
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_02_02-14:06:32] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_02_02-14:06:33] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_02_02-14:06:33] Training with frozen pretrained layers...


2022-02-02 14:06:33.430286: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-02 14:06:34.315854: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-02-02 14:06:36.367547: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/50


2022-02-02 14:06:45.738106: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 7605


11/11 [==============================] - 13s 316ms/step - loss: 0.7434 - val_loss: 0.5545


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/50
11/11 [==============================] - 1s 111ms/step - loss: 0.5535 - val_loss: 0.5220
Epoch 3/50
11/11 [==============================] - 1s 104ms/step - loss: 0.4747 - val_loss: 0.4843
Epoch 4/50
11/11 [==============================] - 1s 102ms/step - loss: 0.4496 - val_loss: 0.4654
Epoch 5/50
11/11 [==============================] - 1s 104ms/step - loss: 0.4369 - val_loss: 0.4678

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 6/50
11/11 [==============================] - 1s 106ms/step - loss: 0.4360 - val_loss: 0.4542
Epoch 7/50
11/11 [==============================] - 1s 105ms/step - loss: 0.4310 - val_loss: 0.4475
Epoch 8/50
11/11 [==============================] - 1s 109ms/step - loss: 0.4273 - val_loss: 0.4496

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
Epoch 9/50
11/11 [==============================] - 1s 105ms/step - loss: 0.4257 - val_loss: 0.4452
Epoch 10/50
11/11 [===================

In [17]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)

In [18]:
print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

Test-set performance:


,# records,AUC
Model seq len,,
512,119,0.917346
All,119,0.917346


Confusion matrix:


,0,1
0,87,9
1,9,14


In [28]:
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

0.6086956521739131

In [26]:
confusion_matrix.loc["1"][1]

14

In [19]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/batch_128_lr_1e-4_2022_02_02.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-02-02 14:15:32.228177: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_128_lr_1e-4_2022_02_02.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_128_lr_1e-4_2022_02_02.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [1]:
import tensorflow as tf
print(tf. __version__)

2.6.2


In [29]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-5
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_02_02-15:34:41] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_02_02-15:34:41] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_02_02-15:34:41] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
11/11 [==============================] - 11s 301ms/step - loss: 0.5757 - val_loss: 0.6380
Epoch 2/50
11/11 [==============================] - 1s 107ms/step - loss: 0.4577 - val_loss: 0.5868
Epoch 3/50
11/11 [==============================] - 1s 105ms/step - loss: 0.4564 - val_loss: 0.4170
Epoch 4/50
11/11 [==============================] - 1s 106ms/step - loss: 0.3867 - val_loss: 0.4442

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 5/50
11/11 [==============================] - 1s 103ms/step - loss: 0.3716 - val_loss: 0.4248

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
Epoch 6/50
11/11 [==============================] - 1s 113ms/step - loss: 0.3668 - val_loss: 0.4062
Epoch 7/50
11/11 [==============================] - 1s 113ms/step - loss: 0.3562 - val_loss: 0.4082

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.
Epoch 8/50
11/11 [==============================] - 1s 

11/11 [==============================] - 11s 439ms/step - loss: 0.3575 - val_loss: 0.4049
Epoch 2/50
11/11 [==============================] - 3s 256ms/step - loss: 0.3539 - val_loss: 0.4036
Epoch 3/50
11/11 [==============================] - 3s 256ms/step - loss: 0.3470 - val_loss: 0.4031
Epoch 4/50
11/11 [==============================] - 3s 256ms/step - loss: 0.3518 - val_loss: 0.4012
Epoch 5/50
11/11 [==============================] - 3s 258ms/step - loss: 0.3431 - val_loss: 0.4008
Epoch 6/50
11/11 [==============================] - 3s 256ms/step - loss: 0.3508 - val_loss: 0.4002
Epoch 7/50
11/11 [==============================] - 3s 255ms/step - loss: 0.3405 - val_loss: 0.4001

Epoch 00007: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
Epoch 8/50
11/11 [==============================] - 3s 262ms/step - loss: 0.3433 - val_loss: 0.4000
Epoch 9/50
11/11 [==============================] - 3s 258ms/step - loss: 0.3462 - val_loss: 0.4000

Epoch 00009: ReduceLROnPlate

21/21 [==============================] - 14s 353ms/step - loss: 0.3691 - val_loss: 0.4045


In [30]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,88,8
1,9,14


0.6222222222222222

In [31]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/batch_128_lr_1e-5_2022_02_02.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_128_lr_1e-5_2022_02_02.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_128_lr_1e-5_2022_02_02.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [32]:
epoch_num = 50
batch_size = 128
learning_rate = 5e-5
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_02_02-15:41:03] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_02_02-15:41:03] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_02_02-15:41:03] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
11/11 [==============================] - 11s 290ms/step - loss: 0.4145 - val_loss: 0.4081
Epoch 2/50
11/11 [==============================] - 1s 103ms/step - loss: 0.3452 - val_loss: 0.4213

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 3/50
11/11 [==============================] - 1s 105ms/step - loss: 0.3442 - val_loss: 0.4044
Epoch 4/50
11/11 [==============================] - 1s 103ms/step - loss: 0.3317 - val_loss: 0.4212

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
Epoch 5/50
11/11 [==============================] - 1s 107ms/step - loss: 0.3417 - val_loss: 0.4024
Epoch 6/50
11/11 [==============================] - 1s 105ms/step - loss: 0.3341 - val_loss: 0.4017
Epoch 7/50
11/11 [==============================] - 1s 110ms/step - loss: 0.3313 - val_loss: 0.4016
Epoch 8/50
11/11 [==============================] - 1s 104ms/step - loss: 0.3231 - val_loss: 0.4016

Epoch 00008: ReduceLROnPlateau reduc

11/11 [==============================] - 11s 441ms/step - loss: 0.3380 - val_loss: 0.4048
Epoch 2/50
11/11 [==============================] - 3s 257ms/step - loss: 0.3285 - val_loss: 0.3971
Epoch 3/50
11/11 [==============================] - 3s 255ms/step - loss: 0.3269 - val_loss: 0.4206

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
Epoch 4/50
11/11 [==============================] - 3s 256ms/step - loss: 0.3235 - val_loss: 0.3989

Epoch 00004: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.
Epoch 5/50
11/11 [==============================] - 3s 257ms/step - loss: 0.3217 - val_loss: 0.3927
Epoch 6/50
11/11 [==============================] - 3s 262ms/step - loss: 0.3130 - val_loss: 0.3917
Epoch 7/50
11/11 [==============================] - 3s 257ms/step - loss: 0.3200 - val_loss: 0.3920

Epoch 00007: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-07.
Epoch 8/50
11/11 [==============================] - 3s 264ms/step - l

21/21 [==============================] - 14s 352ms/step - loss: 0.3435 - val_loss: 0.3930


In [33]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,86,10
1,8,15


0.625

In [34]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/batch_128_lr_5e-5_2022_02_02.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_128_lr_5e-5_2022_02_02.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_128_lr_5e-5_2022_02_02.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
